In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import os

options = webdriver.ChromeOptions()
options.add_experimental_option("prefs", {
  "download.default_directory": r"C:\Users\albin\Downloads\Testes",
  "download.prompt_for_download": False,
  "download.directory_upgrade": True,
  "safebrowsing.enabled": True
})

service = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=service,options=options)

In [3]:
# entrar na página de login (no nosso caso é login.html)
caminho_do_arquivo = os.path.join(os.getcwd(), 'login.html')
navegador.get(caminho_do_arquivo)


In [4]:
# preencher o login e a senha
USER = 'teste1'
PASSWD = 'teste1'

def preenchimento_dados(login, senha):
    preenchimento_campo_user = navegador.find_element(By.XPATH, '/html/body/div/form/input[1]').get_attribute('value')
    preenchimento_campo_pass = navegador.find_element(By.XPATH, '/html/body/div/form/input[2]').get_attribute('value')

    if preenchimento_campo_user!= login:
        navegador.find_element(By.XPATH, '/html/body/div/form/input[1]').clear()
        navegador.find_element(By.XPATH, '/html/body/div/form/input[1]').send_keys(login)

    if preenchimento_campo_pass!=senha:
        navegador.find_element(By.XPATH, '/html/body/div/form/input[2]').clear()
        navegador.find_element(By.XPATH, '/html/body/div/form/input[2]').send_keys(senha)


# clicar no botao de fazer login
def botao_login():
    navegador.find_element(By.XPATH, '/html/body/div/form/button').click()



# Executar o login

preenchimento_dados(USER, PASSWD)
time.sleep(0.5)
botao_login()

In [5]:
# importar a base de clientes
df_notas = pd.read_excel('NotasEmitir.xlsx')

In [6]:
# para cada cliente - rodar o processo de emissao de nota fiscal
for cliente in df_notas.itertuples():
    index, nome, cpf_cnpj, cep, endereco, bairro,municipio, uf, ie, descricao, qtde, valor_unitario, valor_total = cliente
# preencher os dados da NF
    list_inputs = navegador.find_elements(By.TAG_NAME, 'input')
    for i, input_element in enumerate(list_inputs):
    # nome/razao social
        if input_element.get_attribute('name').lower() == 'nome':
            input_element.send_keys(nome)

    # endereco
        if input_element.get_attribute('name').lower() == 'endereco':
            input_element.send_keys(endereco)

    # bairro
        if input_element.get_attribute('name').lower() == 'bairro':
            input_element.send_keys(bairro)
    # municipio
        if input_element.get_attribute('name').lower() == 'municipio':
            input_element.send_keys(municipio)
    # cep
        if input_element.get_attribute('name').lower() == 'cep':
            input_element.send_keys(cep)
    # UF
        elemento = navegador.find_element(By.TAG_NAME, 'select')
        elemento_select = Select(elemento)
        elemento_select.select_by_visible_text(uf)
    # CPF/CNPJ
        if input_element.get_attribute('name').lower() == 'cnpj':
            input_element.send_keys(cpf_cnpj)
    # Inscricao estadual
        if input_element.get_attribute('name').lower() == 'inscricao':
            input_element.send_keys(ie)
    # descrição
        if input_element.get_attribute('name').lower() == 'descricao':
            input_element.send_keys(descricao)
    # quantidade
        if input_element.get_attribute('name').lower() == 'quantidade':
            input_element.send_keys(qtde)
    # valor unitario
        if input_element.get_attribute('name').lower() == 'valor_unitario':
            input_element.send_keys(valor_unitario)
    # valor total
        if input_element.get_attribute('name').lower() == 'total':
            input_element.send_keys(valor_total)
    # clicar em emitir nota fiscal
    navegador.find_element(By.CLASS_NAME,'registerbtn').click()
    # recarregar a página para limpar o formulário
    time.sleep(0.5)
    navegador.refresh()